In [ ]:
### ACTUAL

In [ ]:
import os
import json

def delete_model_jsons(model_name):
    """
    Deletes JSON files for the specified model name across all indicators and countries.
    Replaces spaces in country names with underscores.

    Args:
        model_name (str): The name of the model (e.g., "XGBoost", "Prophet").

    Returns:
        None
    """
    # Load country names and indicators
    with open("../countries.json", "r") as f:
        country_names = json.load(f)
    with open("../indicators.json", "r") as f:
        indicators = json.load(f)

    # Define the base path for the parameter files
    base_dir = "../best_params"

    # Iterate over all indicators and countries to delete JSON files
    for indicator in indicators.keys():
        for country in country_names.keys():
            # Replace spaces in the country name with underscores

            # Construct the file path
            json_file_path = os.path.join(base_dir, indicator, f"{model_name}_{country}.json")
            
            # Check if the file exists and delete it
            if os.path.exists(json_file_path):
                try:
                    os.remove(json_file_path)
                    print(f"Deleted: {json_file_path}")
                except Exception as e:
                    print(f"Error deleting {json_file_path}: {e}")
            else:
                pass

# Example usage:
#delete_model_jsons("LSTM")


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()


def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

from sklearn.model_selection import ParameterGrid

# Hyperparameter tuning for LSTM
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def train_lstm(train, test_y, country, indicator, epochs=50, batch_size=16):
    # Define directories for saving/loading parameters
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"LSTM_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define the hyperparameters grid
        param_grid = {
            'lstm_units': [50, 100],
            'activation': ['relu'],
            'dropout_rate': [0.3, 0.5],
            'optimizer': ['adam'],
            'learning_rate': [0.001, 0.01],
            'sequence_length': [5, 10],
            'batch_size': [16, 32]
        }

        # Dictionary to map optimizer names to optimizer classes
        optimizer_dict = {
            'adam': Adam,
        }

        best_rmse = float('inf')
        best_predictions = None

        # Iterate through all possible hyperparameter combinations
        grid = ParameterGrid(param_grid)
        for params in grid:

            # Create sequences with the current sequence length
            seq_length = params['sequence_length']
            scaler = MinMaxScaler()
            train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
            test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

            X_train, y_train = create_sequences(train_scaled, seq_length)

            # Define LSTM model with current parameters
            model = Sequential([
                LSTM(params['lstm_units'], activation=params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
                Dropout(params['dropout_rate']),
                LSTM(params['lstm_units'], activation=params['activation']),
                Dropout(params['dropout_rate']),
                Dense(1)
            ])

            # Compile model with current optimizer and learning rate
            model.compile(optimizer=optimizer_dict[params['optimizer']](learning_rate=params['learning_rate']), loss='mse')
            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            model.fit(X_train, y_train, epochs=epochs, batch_size=params['batch_size'], verbose=0, callbacks=[early_stop])

            # Rolling prediction on test set
            predictions = []
            input_seq = train_scaled[-seq_length:].tolist()  # Start with last known sequence

            for _ in range(len(test_y)):
                X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
                y_pred = model.predict(X_input, verbose=0).flatten()[0]
                predictions.append(y_pred)
                input_seq.append([y_pred])

            # Convert predictions back to original scale
            predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

            # Calculate RMSE
            rmse = np.sqrt(mean_squared_error(test_y, predictions))

            # Track the best model and parameters
            if rmse < best_rmse:
                best_rmse = rmse
                best_params = params
                best_predictions = predictions

        # Save the best parameters to JSON
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)

    # Prepare data with the best sequence length

    optimizer_dict = {
            'adam': Adam,
        }
    seq_length = best_params['sequence_length']
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

    X_train, y_train = create_sequences(train_scaled, seq_length)

    # Define and compile the final model
    model = Sequential([
        LSTM(best_params['lstm_units'], activation=best_params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(best_params['dropout_rate']),
        LSTM(best_params['lstm_units'], activation=best_params['activation']),
        Dropout(best_params['dropout_rate']),
        Dense(1)
    ])
    model.compile(optimizer=optimizer_dict[best_params['optimizer']](learning_rate=best_params['learning_rate']), loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=epochs, batch_size=best_params['batch_size'], verbose=0, callbacks=[early_stop])

    # Rolling prediction on the test set
    predictions = []
    input_seq = train_scaled[-seq_length:].tolist()

    for _ in range(len(test_y)):
        X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
        y_pred = model.predict(X_input, verbose=0).flatten()[0]
        predictions.append(y_pred)
        input_seq.append([y_pred])

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

    # Save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="LSTM")

    # Return RMSE and predictions
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions



with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                              
                model_errors_rmse[(country, indicator)] = {}

                model_errors_rmse[(country, indicator)]['LSTM'] , lstm_pred = train_lstm(df_original.iloc[:train_size]['Value'], 
                                                                             df_original.iloc[train_size:]['Value'],
                                                                             country,indicator)



                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])

from datetime import datetime
model ="LSMT"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)

c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\c

### TEST

In [2]:
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()


def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

from sklearn.model_selection import ParameterGrid

# Hyperparameter tuning for LSTM
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def train_lstm(train, test_y, country, indicator, epochs=50, batch_size=16):
    # Define directories for saving/loading parameters
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"LSTM_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define the hyperparameters grid
        param_grid = {
            'lstm_units': [50, 100],
            'activation': ['relu'],
            'dropout_rate': [0.3, 0.5],
            'optimizer': ['adam'],
            'learning_rate': [0.001, 0.01],
            'sequence_length': [5, 10],
            'batch_size': [16, 32]
        }

        # Dictionary to map optimizer names to optimizer classes
        optimizer_dict = {
            'adam': Adam,
        }

        best_rmse = float('inf')
        best_predictions = None

        # Iterate through all possible hyperparameter combinations
        grid = ParameterGrid(param_grid)
        for params in grid:

            # Create sequences with the current sequence length
            seq_length = params['sequence_length']
            scaler = MinMaxScaler()
            train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
            test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

            X_train, y_train = create_sequences(train_scaled, seq_length)

            # Define LSTM model with current parameters
            model = Sequential([
                LSTM(params['lstm_units'], activation=params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
                Dropout(params['dropout_rate']),
                LSTM(params['lstm_units'], activation=params['activation']),
                Dropout(params['dropout_rate']),
                Dense(1)
            ])

            # Compile model with current optimizer and learning rate
            model.compile(optimizer=optimizer_dict[params['optimizer']](learning_rate=params['learning_rate']), loss='mse')
            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            model.fit(X_train, y_train, epochs=epochs, batch_size=params['batch_size'], verbose=0, callbacks=[early_stop])

            # Rolling prediction on test set
            predictions = []
            input_seq = train_scaled[-seq_length:].tolist()  # Start with last known sequence

            for _ in range(len(test_y)):
                X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
                y_pred = model.predict(X_input, verbose=0).flatten()[0]
                predictions.append(y_pred)
                input_seq.append([y_pred])

            # Convert predictions back to original scale
            predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

            # Calculate RMSE
            rmse = np.sqrt(mean_squared_error(test_y, predictions))

            # Track the best model and parameters
            if rmse < best_rmse:
                best_rmse = rmse
                best_params = params
                best_predictions = predictions

        # Save the best parameters to JSON
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)

    # Prepare data with the best sequence length

    optimizer_dict = {
            'adam': Adam,
        }
    seq_length = best_params['sequence_length']
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

    X_train, y_train = create_sequences(train_scaled, seq_length)

    # Define and compile the final model
    model = Sequential([
        LSTM(best_params['lstm_units'], activation=best_params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(best_params['dropout_rate']),
        LSTM(best_params['lstm_units'], activation=best_params['activation']),
        Dropout(best_params['dropout_rate']),
        Dense(1)
    ])
    model.compile(optimizer=optimizer_dict[best_params['optimizer']](learning_rate=best_params['learning_rate']), loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=epochs, batch_size=best_params['batch_size'], verbose=0, callbacks=[early_stop])

    # Rolling prediction on the test set
    predictions = []
    input_seq = train_scaled[-seq_length:].tolist()

    for _ in range(len(test_y)):
        X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
        y_pred = model.predict(X_input, verbose=0).flatten()[0]
        predictions.append(y_pred)
        input_seq.append([y_pred])

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

    # Save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="LSTM")

    # Return RMSE and predictions
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions



with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                              
                model_errors_rmse[(country, indicator)] = {}

                model_errors_rmse[(country, indicator)]['LSTM'] , lstm_pred = train_lstm(df_original.iloc[:train_size]['Value'], 
                                                                             df_original.iloc[train_size:]['Value'],
                                                                             country,indicator)



                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])

from datetime import datetime
model ="LSMT"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)

Loading parameters from ../best_params\GDP per Capita (USD)\LSTM_Czech Republic.json...


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


Loading parameters from ../best_params\GDP (USD)\LSTM_Czech Republic.json...


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


Loading parameters from ../best_params\Inflation (CPI)\LSTM_Czech Republic.json...


...

c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\c

No saved parameters found, performing grid search...


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\c

No saved parameters found, performing grid search...


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\c

In [ ]:
### OLDER VERSION

In [ ]:
#LSMT_error_log_2025-02-15--20-19
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()


def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

from sklearn.model_selection import ParameterGrid

# Hyperparameter tuning for LSTM
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def train_lstm(train, test_y, country, indicator, epochs=50, batch_size=16):
    # Define directories for saving/loading parameters
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"LSTM_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define the hyperparameters grid
        param_grid = {
            'lstm_units': [50, 100],
            'activation': ['relu'],
            'dropout_rate': [0.3, 0.5],
            'optimizer': ['adam'],
            'learning_rate': [0.001, 0.01],
            'sequence_length': [5, 10],
            'batch_size': [16, 32]
        }

        # Dictionary to map optimizer names to optimizer classes
        optimizer_dict = {
            'adam': Adam,
        }

        best_rmse = float('inf')
        best_predictions = None

        # Iterate through all possible hyperparameter combinations
        grid = ParameterGrid(param_grid)
        for params in grid:

            # Create sequences with the current sequence length
            seq_length = params['sequence_length']
            scaler = MinMaxScaler()
            train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
            test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

            X_train, y_train = create_sequences(train_scaled, seq_length)

            # Define LSTM model with current parameters
            model = Sequential([
                LSTM(params['lstm_units'], activation=params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
                Dropout(params['dropout_rate']),
                LSTM(params['lstm_units'], activation=params['activation']),
                Dropout(params['dropout_rate']),
                Dense(1)
            ])

            # Compile model with current optimizer and learning rate
            model.compile(optimizer=optimizer_dict[params['optimizer']](learning_rate=params['learning_rate']), loss='mse')
            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            model.fit(X_train, y_train, epochs=epochs, batch_size=params['batch_size'], verbose=0, callbacks=[early_stop])

            # Rolling prediction on test set
            predictions = []
            input_seq = train_scaled[-seq_length:].tolist()  # Start with last known sequence

            for _ in range(len(test_y)):
                X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
                y_pred = model.predict(X_input, verbose=0).flatten()[0]
                predictions.append(y_pred)
                input_seq.append([y_pred])

            # Convert predictions back to original scale
            predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

            # Calculate RMSE
            rmse = np.sqrt(mean_squared_error(test_y, predictions))

            # Track the best model and parameters
            if rmse < best_rmse:
                best_rmse = rmse
                best_params = params
                best_predictions = predictions

        # Save the best parameters to JSON
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)

    # Prepare data with the best sequence length

    optimizer_dict = {
            'adam': Adam,
        }
    seq_length = best_params['sequence_length']
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

    X_train, y_train = create_sequences(train_scaled, seq_length)

    # Define and compile the final model
    model = Sequential([
        LSTM(best_params['lstm_units'], activation=best_params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(best_params['dropout_rate']),
        LSTM(best_params['lstm_units'], activation=best_params['activation']),
        Dropout(best_params['dropout_rate']),
        Dense(1)
    ])
    model.compile(optimizer=optimizer_dict[best_params['optimizer']](learning_rate=best_params['learning_rate']), loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=epochs, batch_size=best_params['batch_size'], verbose=0, callbacks=[early_stop])

    # Rolling prediction on the test set
    predictions = []
    input_seq = train_scaled[-seq_length:].tolist()

    for _ in range(len(test_y)):
        X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
        y_pred = model.predict(X_input, verbose=0).flatten()[0]
        predictions.append(y_pred)
        input_seq.append([y_pred])

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

    # Save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="LSTM")

    # Return RMSE and predictions
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions



with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                              
                model_errors_rmse[(country, indicator)] = {}

                model_errors_rmse[(country, indicator)]['LSTM'] , lstm_pred = train_lstm(df_original.iloc[:train_size]['Value'], 
                                                                             df_original.iloc[train_size:]['Value'],
                                                                             country,indicator)



                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])

from datetime import datetime
model ="LSMT"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)

In [ ]:
# LSMT_error_log_2025-02-11--22-35
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()


def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

def train_lstm(train, test_y, country, indicator, seq_length=5, epochs=50, batch_size=16):
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

    X_train, y_train = create_sequences(train_scaled, seq_length)

    # Define LSTM model
    model = Sequential([
        LSTM(100, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.3),
        LSTM(100, activation='relu'),
        Dropout(0.3),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[early_stop])

    # **Rolling Prediction to Use Entire `test_y`**
    predictions = []
    input_seq = train_scaled[-seq_length:].tolist()  # Start with last known sequence

    for _ in range(len(test_y)):  # Predict for every step in test_y
        X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
        y_pred = model.predict(X_input, verbose=0).flatten()[0]  # Predict next value
        predictions.append(y_pred)  # Store predicted value
        input_seq.append([y_pred])  # Append prediction to sequence for next step

    # Convert predictions back to original scale
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

    # Calculate RMSE using full `test_y`
    rmse = np.sqrt(mean_squared_error(test_y, predictions))

    # Save plot
    save_plot(train, test_y, predictions, country, indicator, model_name="LSTM")
    return rmse , predictions


with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                              
                model_errors_rmse[(country, indicator)] = {}

                model_errors_rmse[(country, indicator)]['LSTM'] , lstm_pred = train_lstm(df_original.iloc[:train_size]['Value'], 
                                                                             df_original.iloc[train_size:]['Value'],
                                                                             country,indicator)



                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])

from datetime import datetime
model ="LSMT"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)